In [1]:
# setup our environment

import pandas as pd
import numpy as np

DATA_FILE = "data/fake_or_real_news.csv"

TEST_SIZE = 0.2
N_SAMPLE = 2000 # we only use N_SAMPLE news stories

In [2]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)
    
    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)
    
    return train_acc, test_acc

In [3]:
# compute 95% confidence interval on error

# NOTE: based on conversation on stackexchange: 
# https://stats.stackexchange.com/questions/247551/how-to-determine-the-confidence-of-a-neural-network-prediction
# towards bottom of the page.

from math import sqrt

def error_conf(error, n):
    term = 1.96*sqrt((error*(1-error))/n)
    lb = error - term
    ub = error + term
    
    return lb, ub

In [4]:
# read in our data and preprocess it
# Note: the news items in the data set range from 0 words to 100,000 words
# we restrict ourselves to news items between 500 and 5,000 words.

df = pd.read_csv(DATA_FILE)
df.drop(labels=['id','title'], axis='columns', inplace=True)
# only select stories with between 500 and 5000 words
mask = list(df['text'].apply(lambda x: len(x) >= 500 and len(x) <= 5000))
df = df[mask]

In [5]:
print(df.shape)

(3510, 2)


In [6]:
# sample down our data set

df = df.sample(n=N_SAMPLE, axis='index')

In [7]:
# show that the sample data is pretty balanced
print(len(df[df['label'] == 'REAL']), len(df[df['label'] == 'FAKE']))

829 1171


In [8]:
X = df['text']
y = df['label']

In [9]:
# convert labels to numeric labels
# NOTE: DNNs need numeric labels

def convert(x):
    if x == 'FAKE':
        return 0.0
    else:
        return 1.0


## Syntactic Features

In [10]:
# set up vector models for training and testing

from sklearn.feature_extraction.text import CountVectorizer

# data vectorizer
vectorizer = CountVectorizer(analyzer = "word", 
                             binary = True, 
                             min_df = 2,
                             stop_words='english')
docarray = vectorizer.fit_transform(X).toarray()
docterm = pd.DataFrame(docarray, columns=vectorizer.get_feature_names())

In [11]:
print(docterm.shape)

(2000, 17793)


In [12]:
# create training and test data
from sklearn.model_selection import train_test_split
docterm_train, docterm_test, y_train, y_test = train_test_split(docterm, y, test_size=TEST_SIZE)

### Naive Bayes

In [13]:
# Naive Bayes Model - train & test
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(docterm_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, docterm_train, y_train, docterm_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 97.31%
Testing Accuracy: 88.50%


In [15]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 85.37%-91.63%


### Random Forest

In [16]:
# Random Forest Model - train & test
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(min_samples_split=60)
model.fit(docterm_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=60,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, docterm_train, y_train, docterm_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 95.88%
Testing Accuracy: 84.25%


In [18]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 80.68%-87.82%


### Deep Neural Network

In [19]:
# DNN
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# build the model
model = Sequential()
model.add(Dense(1024, input_dim=docterm_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(docterm_train, y_train.apply(convert),
          epochs=5,
          batch_size=128,
          validation_data=(docterm_test, y_test.apply(convert)))

Using TensorFlow backend.


Train on 1600 samples, validate on 400 samples
Epoch 1/5
1600/1600 [==============================] - 14s 9ms/step - loss: 0.6992 - acc: 0.5556 - val_loss: 0.5277 - val_acc: 0.8300
Epoch 2/5
1600/1600 [==============================] - 9s 6ms/step - loss: 0.4695 - acc: 0.7987 - val_loss: 0.2681 - val_acc: 0.9025
Epoch 3/5
1600/1600 [==============================] - 9s 6ms/step - loss: 0.2124 - acc: 0.9275 - val_loss: 0.1810 - val_acc: 0.9275
Epoch 4/5
1600/1600 [==============================] - 9s 5ms/step - loss: 0.0434 - acc: 0.9881 - val_loss: 0.2243 - val_acc: 0.9350
Epoch 5/5
1600/1600 [==============================] - 9s 6ms/step - loss: 0.0136 - acc: 0.9969 - val_loss: 0.3038 - val_acc: 0.9375


In [20]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict_classes, 
                                     docterm_train, 
                                     y_train.apply(convert), 
                                     docterm_test, 
                                     y_test.apply(convert))
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 100.00%
Testing Accuracy: 93.75%


In [21]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 91.38%-96.12%


## Semantic Features

In [22]:
# Load Spacy semantic model

import spacy

EMBEDDING_DIM = 300

# NOTE: for performance reasons disable everything in the pipeline except the tokenizer
nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner', 'textcat'])

In [23]:
# preprocess text for semantic features

def embed(X):
    '''
    x is a list of strings and embed will compute
    an embedding vector for each and return an array
    of shape (len(x),EMBEDDING_DIM)
    '''
    vectors = []
    text_array = np.array(X)

    print(text_array.shape)
    
    for i in range(text_array.shape[0]):
        vector = nlp(text_array[i]).vector
        vectors.append(vector)
    
    return pd.DataFrame(vectors)

vectors = embed(X)

(2000,)


In [24]:
# create training and test data
from sklearn.model_selection import train_test_split
vectors_train, vectors_test, y_train, y_test = train_test_split(vectors, y, test_size=TEST_SIZE)

### Naive Bayes

In [25]:
# Naive Bayes Model 
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(vectors_train,y_train)

GaussianNB(priors=None)

In [26]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, vectors_train, y_train, vectors_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 74.00%
Testing Accuracy: 75.75%


In [27]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 71.55%-79.95%


### Random Forest

In [28]:
# Random Forest Model 
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(min_samples_split=10)
model.fit(vectors_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, vectors_train, y_train, vectors_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 98.25%
Testing Accuracy: 82.00%


In [30]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 78.23%-85.77%


### Deep Neural Network

In [31]:
# DNN - MLP
from keras.models import Sequential
from keras.layers import Dense, Dropout

# build the model
model = Sequential()
model.add(Dense(256, input_dim=EMBEDDING_DIM, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [32]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(vectors_train, y_train.apply(convert),
          epochs=15,
          batch_size=128,
          validation_data=(vectors_test, y_test.apply(convert)))

Train on 1600 samples, validate on 400 samples
Epoch 1/15
1600/1600 [==============================] - 4s 2ms/step - loss: 0.6814 - acc: 0.5669 - val_loss: 0.6546 - val_acc: 0.5875
Epoch 2/15
1600/1600 [==============================] - 0s 214us/step - loss: 0.6587 - acc: 0.6006 - val_loss: 0.6060 - val_acc: 0.6750
Epoch 3/15
1600/1600 [==============================] - 0s 264us/step - loss: 0.6233 - acc: 0.6438 - val_loss: 0.5436 - val_acc: 0.7100
Epoch 4/15
1600/1600 [==============================] - 1s 313us/step - loss: 0.5605 - acc: 0.7181 - val_loss: 0.4637 - val_acc: 0.7900
Epoch 5/15
1600/1600 [==============================] - 0s 213us/step - loss: 0.5272 - acc: 0.7450 - val_loss: 0.4498 - val_acc: 0.8025
Epoch 6/15
1600/1600 [==============================] - 0s 228us/step - loss: 0.4747 - acc: 0.7937 - val_loss: 0.4377 - val_acc: 0.8125
Epoch 7/15
1600/1600 [==============================] - 0s 198us/step - loss: 0.4613 - acc: 0.8038 - val_loss: 0.3746 - val_acc: 0.8425
Epo

In [33]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict_classes, 
                                     vectors_train, 
                                     y_train.apply(convert), 
                                     vectors_test, 
                                     y_test.apply(convert))
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 90.38%
Testing Accuracy: 87.25%


In [34]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 83.98%-90.52%
